## Run Example Solvent Extraction Configuration
Install all Install all dependencies using pip install -e .


## Overview
This is a simple test of the solvent extraction code. Same code can be found in front.py

### Import helper modules

In [1]:
""" local imports """
from solventx import solventx as sx
from solventx import utilities as util


### Create and Initialize Solvent Extraction Object
Be sure to update the directory addresses in design.json to match your environment

In [3]:
#
""" build configs """
config_file = ".\\design.json"
confDict = util.read_config(config_file)

config_env_file = ".\\env_design_config.json"
confEnvDict = util.get_env_config_dict(config_env_file)

""" generate cases """
cases = util.generate(confDict,1)

""" select case and create object """
ree_mass = [item for item in cases['0'].values()]
obj = sx.solventx(confDict, confEnvDict, ree_mass)

### Evaluate Simulation and pull example results

In [ ]:
""" evaluate """
obj.evaluate(obj.design_variables) # 

""" Reward """
size = len(ree_mass)
reward = obj.objective(size)

""" Examine Specs """
purity, recovery = obj.constraintfun()
recority = obj.recority # revoery * purity

## For Optimization
 You need to install RBFOPT toolbox, pyomo interface for python integration, and your choice of coin-OR project solvers to handle the MINLP problem

### Helper Function

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""" 
Optimize solvent extraction configuration
"""    
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
def optimize(confDict, confEnvDict, case):
    
    """ instantiate solvent extraction object """
    ree_mass = [item for item in case.values()]
    myb = sx.solventx(confDict, confEnvDict, ree_mass) 

    # Specify optimization settings
    settings = rbfopt.RbfoptSettings(max_evaluations=10, init_strategy='lhd_corr', target_objval=0, max_noisy_restarts=3) #'lhd_corr')#, minlp_solver_path=minlp_solver, nlp_solver_path=nlp_solver) 

    myb.max_iter = settings.max_evaluations    
    alg = rbfopt.RbfoptAlgorithm(settings, myb) 
    rbfopt.RbfoptSettings()

    # call optimizer
    val, x, itercount,evalcount, fast_evalcount = alg.optimize()

    
    """ Get Results """   
    myb.evaluate(x) # Recycle all ree
    
    for i,j in zip(myb.var_space['mutable'], x):    
        print(i, '\t',j)

    
    """ store results in json   """
    result = dict()
    result["rees"] = case
    result["design x"] = [item for item in x]
    result["recovery"] = myb.recovery
    result["purity"] = myb.purity
    
    result["objective"] = {}
    for key,value in myb.recority.items():
        result["objective"][key] = [item for item in value]
    
    result["function value"] = val
    result["constraints"] = myb.constraint
    result["variable space"] = myb.var_space["mutable"]
    result["design"] = {item:jtem for (item,jtem) in zip(myb.var_space["mutable"].keys(),x)}
   
    
    return result


### Run Optimization
The result is returned in the form of a dictionary (see optimize() function above)
Remember to update the addresses to match your local directory system

In [ ]:
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
""" 
Section B: Call the Standard Optimizer
"""    
""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""""
t0 = time.time()

config_file = ".\\design.json"
confDict = util.read_config(config_file)

config_env_file = ".\\env_design_config.json"
confEnvDict = util.get_env_config_dict(config_env_file)

""" generate cases """
cases = util.generate(confDict,3)

t1 = time.time()
#for now, pick only the first cases - later cases will use parallel submission
resjsn = optimize(confDict, confEnvDict, cases['0'])
#

with open('solventx/data/json/results.json', 'w') as json_file:
    json.dump(resjsn, json_file)